In [97]:
#Script created as a light version to directly plot the Statistics Canada (StatCan)
#Physical flow accounts for plastic material (PFAPM) as a Sankey diagram

In [98]:
import pandas as pd
import plotly.graph_objects as go

Sankey of PFAPM by product category

In [99]:
#files used in this script:
    #38100150 (PFAPM by product category)
    #flowmatch
    #product_category_hierarchy
    #product_category_selection

In [100]:
#import raw StatCan PFAPM by product category, as .csv file
#https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3810015001
#or use saved file in 'files' (38100150.csv, downloaded 2024-04-18)
data_raw_StatCan_prod = pd.read_csv("files/38100150.csv",sep=',')
data_raw_StatCan_prod

,REF_DATE,GEO,DGUID,Variable,Product category,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2012,Canada,2021A000011124,Plastic in domestically produced products,"Total, all product categories",Tonnes,287,units,0,v1331406638,1.1.1,3731732.0,NaN,NaN,NaN,0
1,2012,Canada,2021A000011124,Plastic in domestically produced products,Construction materials,Tonnes,287,units,0,v1331406639,1.1.2,1124918.0,NaN,NaN,NaN,0
2,2012,Canada,2021A000011124,Plastic in domestically produced products,Electrical and electronic equipment,Tonnes,287,units,0,v1331406640,1.1.3,182043.0,NaN,NaN,NaN,0
3,2012,Canada,2021A000011124,Plastic in domestically produced products,Electronics,Tonnes,287,units,0,v1331406641,1.1.4,19262.0,NaN,NaN,NaN,0
4,2012,Canada,2021A000011124,Plastic in domestically produced products,Major appliances,Tonnes,287,units,0,v1331406642,1.1.5,40266.0,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17402,2020,Canadian territorial enclaves abroad,NaN,Collected plastic sent directly for disposal o...,Packaging,Tonnes,287,units,0,v1331408504,15.7.7,896.0,NaN,NaN,NaN,0
17403,2020,Canadian territorial enclaves abroad,NaN,Collected plastic sent directly for disposal o...,Agriculture film,Tonnes,287,units,0,v1331408505,15.7.12,0.0,NaN,NaN,NaN,0
17404,2020,Canadian territorial enclaves abroad,NaN,Collected plastic sent directly for disposal o...,Textiles,Tonnes,287,units,0,v1331408506,15.7.13,332.0,NaN,NaN,NaN,0
17405,2020,Canadian territorial enclaves abroad,NaN,Collected plastic sent directly for disposal o...,Vehicles,Tonnes,287,units,0,v1331408507,15.7.14,62.0,NaN,NaN,NaN,0


In [101]:
#create data frame from raw data
df_StatCan_prod=pd.DataFrame(data=data_raw_StatCan_prod)

In [102]:
#import variable matching: 'flowmatch' file (with 'From' and 'To' columns)
#same file for StatCan resin (compiled 2024-05-06)
data_flowmatch_StatCan_prod = pd.read_csv("files/flowmatch.csv", sep=',')
#remove unnecessary columns
data_flowmatch_StatCan_prod=pd.DataFrame(data_flowmatch_StatCan_prod, columns=['Variable','From','To'])
data_flowmatch_StatCan_prod

,Variable,From,To
0,Plastic in domestically produced products,CA domestic production,CA consumption
1,Net trade (imports less exports) of newly prod...,Net trade of new products,CA consumption
2,Plastic in products produced for Canadian cons...,NaN,NaN
3,Net stock of plastic in products that remain i...,CA consumption,Net in-use stock addition
4,Total discarded plastic in products,CA consumption,Total discarded plastics
5,Plastic leaked permanently into the environment,Total discarded plastics,Leak into environment
6,Collected plastic sent directly for disposal o...,Total discarded plastics,Collection
7,Diverted plastic waste and scrap sent for mate...,Collection,Material recovery
8,Disposed plastic waste and scrap not diverted ...,Collection,Total disposed plastics
9,Sorted and baled plastic waste and scrap sent ...,Material recovery,Recycling


In [103]:
#merge data frames (df_StatCan_prod and data_flowmatch_StatCan_prod)
merged_df_StatCan_prod = pd.merge(df_StatCan_prod, data_flowmatch_StatCan_prod, on='Variable', how='outer', validate='many_to_many')
merged_df_StatCan_prod

,REF_DATE,GEO,DGUID,Variable,Product category,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS,From,To
0,2012,Canada,2021A000011124,Collected plastic sent directly for disposal o...,"Total, all product categories",Tonnes,287,units,0,v1331406728,1.7.1,4282879.0,NaN,NaN,NaN,0,Total discarded plastics,Collection
1,2012,Canada,2021A000011124,Collected plastic sent directly for disposal o...,Construction materials,Tonnes,287,units,0,v1331406729,1.7.2,155443.0,NaN,NaN,NaN,0,Total discarded plastics,Collection
2,2012,Canada,2021A000011124,Collected plastic sent directly for disposal o...,Electrical and electronic equipment,Tonnes,287,units,0,v1331406730,1.7.3,396316.0,NaN,NaN,NaN,0,Total discarded plastics,Collection
3,2012,Canada,2021A000011124,Collected plastic sent directly for disposal o...,Electronics,Tonnes,287,units,0,v1331406731,1.7.4,95677.0,NaN,NaN,NaN,0,Total discarded plastics,Collection
4,2012,Canada,2021A000011124,Collected plastic sent directly for disposal o...,Major appliances,Tonnes,287,units,0,v1331406732,1.7.5,24104.0,NaN,NaN,NaN,0,Total discarded plastics,Collection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17402,2020,Nunavut,2021A000262,Total disposed plastic waste and scrap,Packaging,Tonnes,287,units,0,v1331408448,14.15.7,1125.0,NaN,NaN,NaN,0,NaN,NaN
17403,2020,Nunavut,2021A000262,Total disposed plastic waste and scrap,Agriculture film,Tonnes,287,units,0,v1331408449,14.15.12,0.0,NaN,NaN,NaN,0,NaN,NaN
17404,2020,Nunavut,2021A000262,Total disposed plastic waste and scrap,Textiles,Tonnes,287,units,0,v1331408450,14.15.13,212.0,NaN,NaN,NaN,0,NaN,NaN
17405,2020,Nunavut,2021A000262,Total disposed plastic waste and scrap,Vehicles,Tonnes,287,units,0,v1331408451,14.15.14,850.0,NaN,NaN,NaN,0,NaN,NaN


In [104]:
#convert into multi-index to filter for relevant columns and rows
df_StatCan_prod_multiind=merged_df_StatCan_prod.set_index(['REF_DATE','GEO','Variable','Product category'],inplace=False)
df_StatCan_prod_multiind

DGUID  \
REF_DATE GEO     Variable                                           Product category                                      
2012     Canada  Collected plastic sent directly for disposal or... Total, all product categories        2021A000011124   
                                                                    Construction materials               2021A000011124   
                                                                    Electrical and electronic equipment  2021A000011124   
                                                                    Electronics                          2021A000011124   
                                                                    Major appliances                     2021A000011124   
...                                                                                                                 ...   
2020     Nunavut Total disposed plastic waste and scrap             Packaging                               2021A000262   
                                                                    Agriculture film                        2021A000262   
                                                                    Textiles                                2021A000262   
                                                                    Vehicles                                2021A000262   
                                                                    Other products                          2021A000262   

                                                                                                            UOM  \
REF_DATE GEO     Variable                                           Product category                              
2012     Canada  Collected plastic sent directly for disposal or... Total, all product categories        Tonnes   
                                                                    Construction materials               Tonnes   
                                                                    Electrical and electronic equipment  Tonnes   
                                                                    Electronics                          Tonnes   
                                                                    Major appliances                     Tonnes   
...                                                                                                         ...   
2020     Nunavut Total disposed plastic waste and scrap             Packaging                            Tonnes   
                                                                    Agriculture film                     Tonnes   
                                                                    Textiles                             Tonnes   
                                                                    Vehicles                             Tonnes   
                                                                    Other products                       Tonnes   

                                                                                                         UOM_ID  \
REF_DATE GEO     Variable                                           Product category                              
2012     Canada  Collected plastic sent directly for disposal or... Total, all product categories           287   
                                                                    Construction materials                  287   
                                                                    Electrical and electronic equipment     287   
                                                                    Electronics                             287   
                                                                    Major appliances                        287   
...                                                                                                         ...   
2020     Nunavut Total disposed plastic waste and scrap             Packaging                     

In [105]:
#select year ("REF_DATE") and geography ("GEO") you want to plot
#here REF_DATE = 2020, and GEO = Canada
df_StatCan_prod_2020_CA = df_StatCan_prod_multiind.loc[2020,'Canada',:,:]
df_StatCan_prod_2020_CA

DGUID  \
Variable                                           Product category                                      
Collected plastic sent directly for disposal or... Total, all product categories        2021A000011124   
                                                   Construction materials               2021A000011124   
                                                   Electrical and electronic equipment  2021A000011124   
                                                   Electronics                          2021A000011124   
                                                   Major appliances                     2021A000011124   
...                                                                                                ...   
Total disposed plastic waste and scrap             Other packaging products             2021A000011124   
                                                   Agriculture film                     2021A000011124   
                                                   Textiles                             2021A000011124   
                                                   Vehicles                             2021A000011124   
                                                   Other products                       2021A000011124   

                                                                                           UOM  \
Variable                                           Product category                              
Collected plastic sent directly for disposal or... Total, all product categories        Tonnes   
                                                   Construction materials               Tonnes   
                                                   Electrical and electronic equipment  Tonnes   
                                                   Electronics                          Tonnes   
                                                   Major appliances                     Tonnes   
...                                                                                        ...   
Total disposed plastic waste and scrap             Other packaging products             Tonnes   
                                                   Agriculture film                     Tonnes   
                                                   Textiles                             Tonnes   
                                                   Vehicles                             Tonnes   
                                                   Other products                       Tonnes   

                                                                                        UOM_ID  \
Variable                                           Product category                              
Collected plastic sent directly for disposal or... Total, all product categories           287   
                                                   Construction materials                  287   
                                                   Electrical and electronic equipment     287   
                                                   Electronics                             287   
                                                   Major appliances                        287   
...                                                                                        ...   
Total disposed plastic waste and scrap             Other packaging products                287   
                                                   Agriculture film                        287   
                                                   Textiles                                287   
                                                   Vehicles                                287   
                                                   Other products                          287   

                                                                                       SCALAR_FACTOR  \
Variable                                           Product categ

In [106]:
#reset index of df_StatCan_prod_2020_CA
df_StatCan_prod_2020_CA = df_StatCan_prod_2020_CA.reset_index()

#turn 'VALUE' into absolute values
df_StatCan_prod_2020_CA['VALUE'] =df_StatCan_prod_2020_CA['VALUE'].apply(lambda x: abs(x))
df_StatCan_prod_2020_CA

,Variable,Product category,DGUID,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS,From,To
0,Collected plastic sent directly for disposal o...,"Total, all product categories",2021A000011124,Tonnes,287,units,0,v1331406728,1.7.1,4887289.0,NaN,NaN,NaN,0,Total discarded plastics,Collection
1,Collected plastic sent directly for disposal o...,Construction materials,2021A000011124,Tonnes,287,units,0,v1331406729,1.7.2,164611.0,NaN,NaN,NaN,0,Total discarded plastics,Collection
2,Collected plastic sent directly for disposal o...,Electrical and electronic equipment,2021A000011124,Tonnes,287,units,0,v1331406730,1.7.3,439203.0,NaN,NaN,NaN,0,Total discarded plastics,Collection
3,Collected plastic sent directly for disposal o...,Electronics,2021A000011124,Tonnes,287,units,0,v1331406731,1.7.4,120034.0,NaN,NaN,NaN,0,Total discarded plastics,Collection
4,Collected plastic sent directly for disposal o...,Major appliances,2021A000011124,Tonnes,287,units,0,v1331406732,1.7.5,25661.0,NaN,NaN,NaN,0,Total discarded plastics,Collection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Total disposed plastic waste and scrap,Other packaging products,2021A000011124,Tonnes,287,units,0,v1331406858,1.15.11,89328.0,NaN,NaN,NaN,0,NaN,NaN
221,Total disposed plastic waste and scrap,Agriculture film,2021A000011124,Tonnes,287,units,0,v1331406859,1.15.12,36706.0,NaN,NaN,NaN,0,NaN,NaN
222,Total disposed plastic waste and scrap,Textiles,2021A000011124,Tonnes,287,units,0,v1331406860,1.15.13,275695.0,NaN,NaN,NaN,0,NaN,NaN
223,Total disposed plastic waste and scrap,Vehicles,2021A000011124,Tonnes,287,units,0,v1331406861,1.15.14,721398.0,NaN,NaN,NaN,0,NaN,NaN


In [107]:
#import product_category_hierarchy file (compiled 2024-05-06)
data_hierarchy_StatCan_prod = pd.read_csv("files/product_category_hierarchy.csv",sep=',')
data_hierarchy_StatCan_prod

,Level,Product category
0,1,"Total, all product categories"
1,2,Construction materials
2,2,Electrical and electronic equipment
3,3,Electronics
4,3,Major appliances
5,3,Other electrical and electronic equipment
6,2,Packaging
7,3,Bottles
8,3,Film
9,3,Non-bottle rigid


In [108]:
#merge data frames: df_StatCan_prod_2020_CA and data_hierarchy_StatCan_prod
df_StatCan_prod_2020_CA = pd.merge(df_StatCan_prod_2020_CA, data_hierarchy_StatCan_prod, on='Product category', how='outer',validate='many_to_many')
df_StatCan_prod_2020_CA

,Variable,Product category,DGUID,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS,From,To,Level
0,Collected plastic sent directly for disposal o...,Agriculture film,2021A000011124,Tonnes,287,units,0,v1331406739,1.7.12,43891.0,NaN,NaN,NaN,0,Total discarded plastics,Collection,2
1,Disposed plastic waste and scrap not diverted ...,Agriculture film,2021A000011124,Tonnes,287,units,0,v1331406769,1.9.12,33699.0,NaN,NaN,NaN,0,Collection,Total disposed plastics,2
2,Diverted plastic waste and scrap sent for mate...,Agriculture film,2021A000011124,Tonnes,287,units,0,v1331406754,1.8.12,10191.0,NaN,NaN,NaN,0,Collection,Material recovery,2
3,Net stock of plastic in products that remain i...,Agriculture film,2021A000011124,Tonnes,287,units,0,v1331406694,1.4.12,8863.0,NaN,NaN,NaN,0,CA consumption,Net in-use stock addition,2
4,Net trade (imports less exports) of newly prod...,Agriculture film,2021A000011124,Tonnes,287,units,0,v1331406664,1.2.12,7770.0,NaN,NaN,NaN,0,Net trade of new products,CA consumption,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Residual plastic waste and scrap sent for disp...,Vehicles,2021A000011124,Tonnes,287,units,0,v1331406846,1.14.14,1114.0,NaN,NaN,NaN,0,Recycling,Total disposed plastics,2
221,Residual plastic waste and scrap sent for disp...,Vehicles,2021A000011124,Tonnes,287,units,0,v1331406801,1.11.14,720284.0,NaN,NaN,NaN,0,Material recovery,Total disposed plastics,2
222,Sorted and baled plastic waste and scrap sent ...,Vehicles,2021A000011124,Tonnes,287,units,0,v1331406786,1.10.14,7276.0,NaN,NaN,NaN,0,Material recovery,Recycling,2
223,Total discarded plastic in products,Vehicles,2021A000011124,Tonnes,287,units,0,v1331406711,1.5.14,732599.0,NaN,NaN,NaN,0,CA consumption,Total discarded plastics,2


In [109]:
#Set level of end-of-life flows to 'Level' = 2 (even though the data is only for 'Total, all resins')
#The end-of-life flows considered are:
    #'Disposed plastic waste and scrap sent to landfill or incinerated without energy recovery'
    #'Disposed plastic waste and scrap sent for incineration or gasification with energy recovery'
    #'Net trade (imports less exports) of disposed plastic waste and scrap'
df_StatCan_prod_2020_CA.loc[df_StatCan_prod_2020_CA['Variable'] 
== 'Disposed plastic waste and scrap sent to landfill or incinerated without energy recovery','Level']=2

df_StatCan_prod_2020_CA.loc[df_StatCan_prod_2020_CA['Variable'] 
== 'Disposed plastic waste and scrap sent for incineration or gasification with energy recovery','Level']=2

df_StatCan_prod_2020_CA.loc[df_StatCan_prod_2020_CA['Variable'] 
== 'Net trade (imports less exports) of disposed plastic waste and scrap','Level']=2
#exclude all other ('Level' = 1,3)
df_StatCan_prod_2020_CA_2 = df_StatCan_prod_2020_CA[df_StatCan_prod_2020_CA['Level']== 2]
df_StatCan_prod_2020_CA_2

,Variable,Product category,DGUID,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS,From,To,Level
0,Collected plastic sent directly for disposal o...,Agriculture film,2021A000011124,Tonnes,287,units,0,v1331406739,1.7.12,43891.0,NaN,NaN,NaN,0,Total discarded plastics,Collection,2
1,Disposed plastic waste and scrap not diverted ...,Agriculture film,2021A000011124,Tonnes,287,units,0,v1331406769,1.9.12,33699.0,NaN,NaN,NaN,0,Collection,Total disposed plastics,2
2,Diverted plastic waste and scrap sent for mate...,Agriculture film,2021A000011124,Tonnes,287,units,0,v1331406754,1.8.12,10191.0,NaN,NaN,NaN,0,Collection,Material recovery,2
3,Net stock of plastic in products that remain i...,Agriculture film,2021A000011124,Tonnes,287,units,0,v1331406694,1.4.12,8863.0,NaN,NaN,NaN,0,CA consumption,Net in-use stock addition,2
4,Net trade (imports less exports) of newly prod...,Agriculture film,2021A000011124,Tonnes,287,units,0,v1331406664,1.2.12,7770.0,NaN,NaN,NaN,0,Net trade of new products,CA consumption,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Residual plastic waste and scrap sent for disp...,Vehicles,2021A000011124,Tonnes,287,units,0,v1331406846,1.14.14,1114.0,NaN,NaN,NaN,0,Recycling,Total disposed plastics,2
221,Residual plastic waste and scrap sent for disp...,Vehicles,2021A000011124,Tonnes,287,units,0,v1331406801,1.11.14,720284.0,NaN,NaN,NaN,0,Material recovery,Total disposed plastics,2
222,Sorted and baled plastic waste and scrap sent ...,Vehicles,2021A000011124,Tonnes,287,units,0,v1331406786,1.10.14,7276.0,NaN,NaN,NaN,0,Material recovery,Recycling,2
223,Total discarded plastic in products,Vehicles,2021A000011124,Tonnes,287,units,0,v1331406711,1.5.14,732599.0,NaN,NaN,NaN,0,CA consumption,Total discarded plastics,2


In [110]:
#import product category colour selection file (product_category_selection, compiled 2024-09-13)
data_selection_StatCan_prod = pd.read_csv("files/product_category_selection.csv",sep=',')
data_selection_StatCan_prod

,Level,Product category,Colour_HEX,Colour
0,1,"Total, all product categories",#7F7F7F,grey
1,2,Construction materials,#00A1C0,turqoise
2,2,Electrical and electronic equipment,#FA961E,orange
3,3,Electronics,#FEE9A2,light-yellow
4,3,Major appliances,#FEE9A2,light-yellow
5,3,Other electrical and electronic equipment,#FEE9A2,light-yellow
6,2,Packaging,#0BAC43,green
7,3,Bottles,#47F383,light-green
8,3,Film,#47F383,light-green
9,3,Non-bottle rigid,#47F383,light-green


In [111]:
#merge data frames: df_StatCan_prod_2020_CA_2 and data_selection_StatCan_prod
df_StatCan_prod_2020_CA_2 = pd.merge(df_StatCan_prod_2020_CA_2, data_selection_StatCan_prod, on='Product category', how='outer', validate='many_to_many')
df_StatCan_prod_2020_CA_2

,Variable,Product category,DGUID,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS,From,To,Level_x,Level_y,Colour_HEX,Colour
0,Collected plastic sent directly for disposal o...,Agriculture film,2021A000011124,Tonnes,287.0,units,0.0,v1331406739,1.7.12,43891.0,NaN,NaN,NaN,0.0,Total discarded plastics,Collection,2.0,2,#B91E32,red
1,Disposed plastic waste and scrap not diverted ...,Agriculture film,2021A000011124,Tonnes,287.0,units,0.0,v1331406769,1.9.12,33699.0,NaN,NaN,NaN,0.0,Collection,Total disposed plastics,2.0,2,#B91E32,red
2,Diverted plastic waste and scrap sent for mate...,Agriculture film,2021A000011124,Tonnes,287.0,units,0.0,v1331406754,1.8.12,10191.0,NaN,NaN,NaN,0.0,Collection,Material recovery,2.0,2,#B91E32,red
3,Net stock of plastic in products that remain i...,Agriculture film,2021A000011124,Tonnes,287.0,units,0.0,v1331406694,1.4.12,8863.0,NaN,NaN,NaN,0.0,CA consumption,Net in-use stock addition,2.0,2,#B91E32,red
4,Net trade (imports less exports) of newly prod...,Agriculture film,2021A000011124,Tonnes,287.0,units,0.0,v1331406664,1.2.12,7770.0,NaN,NaN,NaN,0.0,Net trade of new products,CA consumption,2.0,2,#B91E32,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,Residual plastic waste and scrap sent for disp...,Vehicles,2021A000011124,Tonnes,287.0,units,0.0,v1331406846,1.14.14,1114.0,NaN,NaN,NaN,0.0,Recycling,Total disposed plastics,2.0,2,#FDDA64,yellow
108,Residual plastic waste and scrap sent for disp...,Vehicles,2021A000011124,Tonnes,287.0,units,0.0,v1331406801,1.11.14,720284.0,NaN,NaN,NaN,0.0,Material recovery,Total disposed plastics,2.0,2,#FDDA64,yellow
109,Sorted and baled plastic waste and scrap sent ...,Vehicles,2021A000011124,Tonnes,287.0,units,0.0,v1331406786,1.10.14,7276.0,NaN,NaN,NaN,0.0,Material recovery,Recycling,2.0,2,#FDDA64,yellow
110,Total discarded plastic in products,Vehicles,2021A000011124,Tonnes,287.0,units,0.0,v1331406711,1.5.14,732599.0,NaN,NaN,NaN,0.0,CA consumption,Total discarded plastics,2.0,2,#FDDA64,yellow


In [112]:
#reduce data frame to only 'VALUE', 'From', 'To', and 'Colour_HEX' that are needed for the Sankey
links = pd.DataFrame(df_StatCan_prod_2020_CA_2, columns=['VALUE','From','To','Colour_HEX'])

#remove rows where 'From' (and 'To') = "NaN"
links = links.dropna(subset=['From'])
links

,VALUE,From,To,Colour_HEX
0,43891.0,Total discarded plastics,Collection,#B91E32
1,33699.0,Collection,Total disposed plastics,#B91E32
2,10191.0,Collection,Material recovery,#B91E32
3,8863.0,CA consumption,Net in-use stock addition,#B91E32
4,7770.0,Net trade of new products,CA consumption,#B91E32
...,...,...,...,...
106,6161.0,Recycling,Secondary feedstock,#FDDA64
107,1114.0,Recycling,Total disposed plastics,#FDDA64
108,720284.0,Material recovery,Total disposed plastics,#FDDA64
109,7276.0,Material recovery,Recycling,#FDDA64


In [124]:
#create unique list of source/'From' and target/'To' for plotly
unique_from_to = list(pd.unique(links[['From','To']].values.ravel('K')))

# Create a list of unique nodes
unique_nodes = pd.concat([links['From'], links['To']]).unique()
node_index = {node: i for i, node in enumerate(unique_nodes)}

# Create new columns for indices in a safe way
links = links.assign(
    FromIndex=links['From'].map(node_index),
    ToIndex=links['To'].map(node_index))

#plot plotly Sankey
fig = go.Figure(data=[go.Sankey(
    valueformat = None,
    valuesuffix = 't', #all flows in unit tonnes
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = unique_from_to,
      color = "#7F7F7F"
    ),
    link = dict(
      source = links['FromIndex'], # indices correspond to labels
      target = links['ToIndex'],
      value = links['VALUE'],
        color = links['Colour_HEX']
  ))])

# Create legend
legend_trace = []
#creat two lists for legend of product categories and colour
categories= data_selection_StatCan_prod['Product category']
colors= data_selection_StatCan_prod['Colour_HEX']
#remove not-considered sub-categories of Electrical and electronic equipment (Electronics, Major appliances, Other electrical and electronic equipment)
#and of Packaging (Bottles, Film, Non-bottle rigid, Other packaging products)
    #remove index 3,4,5 and 7,8,9,10
categories= categories.drop([3,4,5,7,8,9,10])
colors= colors.drop([3,4,5,7,8,9,10])
for idx, (cat, color) in enumerate(zip(categories, colors)):
    legend_trace.append(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=10, color=color),
        name=cat
    ))

# Add legend to the figure
for trace in legend_trace:
    fig.add_trace(trace)

fig.update_layout(
    title_text="Sankey Diagram, PFAPM by product category, 2020, Canada", 
    font_size=10,
    font_color = "black",
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    showlegend=True,
    legend=dict(
        title='Legend',
        orientation='h',
        yanchor='top',
        y=1.1,
        xanchor='right',
        x=0.9),
    width=1920,
    height=700,
    font=dict(size=16),
    plot_bgcolor='rgba(0,0,0,0)',     # Transparent plot area
    paper_bgcolor='rgba(0, 0, 0, 0)', # Transparent figure background
)
fig.show()

In [125]:
#save figure in html format in files (or results or other) folder
fig.write_html("files/Sankey_prod_CA_2020.html")